In [1]:
from extraction import *
from read_edf import *
from read_result import *
from split_signal import *
from processing_data import *
import numpy as np

下面这段是接口的罗列，一组30s的EEG信号可以处理成35*3组特征

In [2]:
edf_path = "../database/sleep-cassette/SC4001E0-PSG.edf"
result_path = "../database/sleep-cassette/SC4001EC-Hypnogram.edf"

In [3]:
start_times, durations, labels = read_result_file(result_path)
res = split_edf_by_annotations(edf_path, start_times, durations, labels)
split_signals = split_edf_by_30s(res)
for (_, signals) in split_signals:
    for i in range(3):
        signals[i] = signal_filter(signals[i])

In [4]:
calculate_zero_crossings(signals[0])

289

In [5]:
calculate_hjorth_parameters(signals[0])

(0.24802843204829975, 4.442511638348952)

In [6]:
np.std(signals[0])

22.787917537730344

In [7]:
calculate_binned_entropy(signals[0], 5)

1.6309753794369857

Page 6, Table 1:

- The binned entropy feature first bins the time series to then sum up the entropy of the bins (this feature is calculated four times for each time a different number of bins; 5, 10, 30, and 60 bins)

- The binned Fourier entropy feature calculates the binned entropy of the power spectral density (this feature is calculated for seven different bin sizes, i.e., 2, 3, 5, 10, 30, 60, and 100 bins)

In [8]:
time_entropy_bins = [5, 10, 30, 60]
frequency_entropy_bins = [2, 3, 5, 10, 30, 60, 100]

In [9]:
frequency_signals = perform_fft(signals[0], 100)
frequency_signals
# print(frequency_signals.shape)

array([448.9615648  -0.j        , 450.6553505 -27.035291j  ,
       455.82845164-54.92775882j, ..., 464.87657357+84.60397162j,
       455.82845164+54.92775882j, 450.6553505 +27.035291j  ])

In [10]:
res = calculate_frequency_domain_features(frequency_signals, 100)

In [11]:
res

{'spectral_centroid': -0.7961957451536072,
 'spectral_variance': 852.346368245573,
 'spectral_skewness': 1.2915924313178013,
 'spectral_kurtosis': 1.2751109534897003,
 'binned_entropies': [0.16689582822799154,
  0.384645141124396,
  0.7214050142113944,
  1.3043946344721347,
  2.500929582639492,
  3.338748004663165,
  3.9987252329249223],
 'band_powers': {'delta_slow': 94816340737.24715,
  'delta_fast': 218092675313.38623,
  'theta': 723001451147.9615,
  'alpha': 916415342758.6603,
  'sigma': 78102220933.33379,
  'beta': 3531057243035.98},
 'power_ratios': {'fast_delta_theta': 941094126461.3478,
  'alpha_theta': 1.267515219096174,
  'delta_beta': 0.02685211091500875,
  'delta_sigma': 1.2140031308223633,
  'delta_theta': 0.13114267002742583}}

In [12]:
res = signal_extraction(signals[0])

In [13]:
res.shape

(35,)

下面是批量处理部分

In [14]:
import os
from tqdm import tqdm
import time

In [15]:
edf_files = []
result_files = []

for dirname, _, filenames in os.walk('../database/sleep-cassette'):
    for filename in filenames:
        if filename[9] == 'P':
            edf_files.append(os.path.join(dirname, filename))
        else:
            result_files.append(os.path.join(dirname, filename))
N = len(result_files)
print(N)

X_list = []
Y_list = []

label_dir = {
    'Sleep stage W': 0,
    'Sleep stage 1': 1,
    'Sleep stage 2': 2,
    'Sleep stage 3': 3,
    'Sleep stage 4': 4,
    'Sleep stage R': 5,
    'Sleep stage ?': 6
}

for i in range(61,121):
    result_path = result_files[i]
    edf_path = edf_files[i]
    extract_single_file(edf_path, result_path)


153


SC4311E0: 100%|██████████| 2531/2531 [03:01<00:00, 13.97it/s]


done: SC4311E0


SC4312E0: 100%|██████████| 2449/2449 [02:46<00:00, 14.69it/s]


done: SC4312E0


...

done: SC4612E0


SC4621E0: 100%|██████████| 2423/2423 [04:01<00:00, 10.02it/s]


done: SC4621E0


SC4622E0: 100%|██████████| 2661/2661 [03:00<00:00, 14.76it/s]

done: SC4622E0
